## Trabalho Prático 3 - Exploração e Mapeamento

### Importando as bibliotecas

In [1]:
import numpy as np
from pyparsing import line
import random as rd
import matplotlib.pyplot as plt
from RobotController import RobotController
from PathPlanners import tt_force
from skimage.draw import line
import time

from coppeliasim_zmqremoteapi_client import *


## Remote API functions (Python)

Criação do cliente para conexão com a api remota

Link para repositório da SimZMQRemoteApi: https://github.com/CoppeliaRobotics/zmqRemoteApi/tree/master/clients/python


In [2]:
# create a client to connect to zmqRemoteApi server:
# (creation arguments can specify different host/port,
# defaults are host='localhost', port=23000)
client = RemoteAPIClient()

# get a remote object:
sim = client.require('sim')

# call API function fo test:
Rack = sim.getObject('/kobuki')
print("Printando algo em cena: " + str(Rack))

Printando algo em cena: 158


### Leitura dos sensores

In [3]:
def readSensorData(range_data_signal_id="hokuyo_range_data", 
                    angle_data_signal_id="hokuyo_angle_data"):
    """
    Reads sensor data from the specified signal IDs.

    Parameters:
    - range_data_signal_id (str): The signal ID for the range data.
    - angle_data_signal_id (str): The signal ID for the angle data.

    Returns:
    - raw_range_data (numpy.ndarray): The unpacked range data.
    - raw_angle_data (numpy.ndarray): The unpacked angle data.
    """
    
    string_range_data = sim.getStringSignal(range_data_signal_id)

    string_angle_data = sim.getStringSignal(angle_data_signal_id)

    # verifique se ambos os dados foram obtidos corretamente
    if string_range_data != None and string_angle_data != None:
        # descompacte dados de mensagens de alcance e sensor
        raw_range_data = sim.unpackFloatTable(string_range_data)
        raw_angle_data = sim.unpackFloatTable(string_angle_data)

        return raw_range_data, raw_angle_data
    else:
        return np.array([]), np.array([])

def get_all_laser_data_ruido(laser_data, trans_global):
    """
    Applies noise to laser data and transforms it to global coordinates.

    Args:
        laser_data (list): List of tuples representing laser data in polar coordinates.
                           Each tuple contains an angle and distance.
        trans_global (numpy.ndarray): Transformation matrix for converting laser data to global coordinates.

    Returns:
        list: List of points in global coordinates with added noise.

    """
    dados_ruidos = []
    for ang, dist in laser_data:
        if dist < 5:  # Considerando leituras válidas do laser
            x = dist * np.cos(ang) + rd.uniform(-0.05, 0.05)  # Adicionando ruído na coordenada x
            y = dist * np.sin(ang) + rd.uniform(-0.05, 0.05)  # Adicionando ruído na coordenada y
            ponto_global = trans_global @ np.array([x, y, 1])
            dados_ruidos.append([ponto_global[0], ponto_global[1]])
    return dados_ruidos
    

### Funções Auxiliares

In [4]:
# Função para rotacionar em torno do eixo z
def Rz(theta):
    return np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])

def get_object_position(sim, object_name):
    return sim.getObjectPosition(sim.getObject(object_name), -1)

def get_object_orientation(sim, object_name):
    return sim.getObjectOrientation(sim.getObject(object_name), -1)

# Funções de transformação e conversão
def transformacao(robot_pos, robot_ori):
    """
    Calcula a matriz de transformação homogénea para uma dada posição e orientação do robô.

    Parameters:
    robot_pos (list): The position of the robot in the form [x, y, z].
    robot_ori (list): The orientation of the robot in the form [roll, pitch, yaw].

    Returns:
    numpy.ndarray: The homogeneous transformation matrix.

    """
    # Matriz de rotação em Z
    c, s = np.cos(robot_ori[2]), np.sin(robot_ori[2])
    Rz = np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])
    
    # Matriz de translação
    T = np.array([robot_pos[0], robot_pos[1], 0])
    
    # Matriz de transformação homogênea
    H = np.eye(4)
    H[:3, :3] = Rz
    H[:3, 3] = T
    return H

def world_to_grid(world_point, map_size, cell_size):
    """
    Converte coordenadas do mundo para coordenadas do grid
    
    Parâmetros:
    world_point (np.array): Coordenadas do mundo
    map_size (int): Tamanho do mapa
    cell_size (int): Tamanho da célula do grid
    """
    grid_point = np.floor((world_point[:2] + map_size / 2) / cell_size).astype(int)
    return grid_point

### Occupancy Grid 

In [5]:
# Função para calcular o log odds a partir da probabilidade
def log_odds(prob):
    return np.log(prob / (1 - prob))

# Função para calcular a probabilidade a partir do log odds
def inv_log_odds(lo):
    return 1 - 1 / (1 + np.exp(lo))

# Cálculo do inverse sensor model
def inverse_sensor_model(m_i, x_t, z_t, l0):
    """
    Calcula o modelo de sensor inverso para uma célula específica.

    Parâmetros:
    - m_i: Coordenadas da célula (numpy array)
    - x_t: Posição do robô (numpy array)
    - z_t: Distância medida pelo sensor
    - l0: Valor inicial de log odds

    Retorna:
    - Atualização de log odds para a célula m_i
    """
    max_distance = 5 / cell_size
    alpha = 0.2 / cell_size
    distance_to_cell = np.linalg.norm(m_i - x_t[:2])
    p_occ = 0.85
    p_free = 0.1
    l_occ = log_odds(p_occ)
    l_free = log_odds(p_free)
    
    if distance_to_cell > min(max_distance, z_t + alpha / 2): # Se a célula está fora do alcance do sensor
        return log_odds(l0) # Retorna o valor inicial de log odds
    if z_t < max_distance and abs(distance_to_cell - z_t) <= alpha / 2: # Se a célula está ocupada
        return l_occ # Retorna o log odds de ocupação
    if distance_to_cell <= z_t: # Se a célula está livre
        return l_free # Retorna o log odds de célula livre
    
    return log_odds(l0) # Retorna o valor inicial de log odds

def occupancy_grid_mapping(x_t, z_t, occupancy_grid):
    """
    Atualiza o occupancy grid com base nos dados do sensor.

    Parâmetros:
    - occupancy_grid: Grid de ocupação (numpy array)
    - x_t: Posição do robô (numpy array)
    - z_t: Posição do ponto final detectado pelo sensor (numpy array)
    - l0: Valor inicial de log odds

    Retorna:
    - occupancy_grid: Grid de ocupação atualizado
    """
    max_distance = np.linalg.norm(x_t - z_t)
    rr, cc = line(x_t[0], x_t[1], z_t[0], z_t[1])
    for i in range(len(rr)):
        cel_prob = occupancy_grid[rr[i], cc[i]]
        l_ant = log_odds(cel_prob)
        m_i = np.array([rr[i], cc[i]])
        l_cell = l_ant + inverse_sensor_model(m_i, x_t, max_distance, cel_prob)
        occupancy_grid[rr[i], cc[i]] = inv_log_odds(l_cell)
        #print(f"Updated cell [{rr[i]}, {cc[i]}] to {occupancy_grid[rr[i], cc[i]]}")
    return occupancy_grid


### Navegação

In [6]:
# Criando o grid
map_size = np.array([10, 10])
cell_size = 0.1
rows, cols = (map_size / cell_size).astype(int)

# Inicializando o grid
occupancy_grid = np.full((rows, cols), 0.5)

# Conectando-se ao CoppeliaSim
clientID = sim.startSimulation()

if clientID != -1:
    print('Program Started')
    
    robot_controller = RobotController('kobuki')
    robot_controller.start_simulation()
    
    goal_positions = [[0.4, -4.45, 1], [0.8, -1.8, 1], [1.7, 0, 1], [0.7, 3, 1], [2.5, 4.3, 1]] 
    laser_range_data = "hokuyo_range_data"
    laser_angle_data = "hokuyo_angle_data"
    
    laser_points = []
    cont = 0

    while cont < len(goal_positions):
        print('Goal position:', goal_positions[cont], 'Contador: ', cont)
        goal_position = goal_positions[cont]
        robot_controller.set_goal_position(goal_position) 

        while True:
            robot_pos = robot_controller.get_robot_position()
            robot_ori = robot_controller.get_robot_orientation()

            distance_goal = np.linalg.norm(np.array(goal_position[:2]) - np.array(robot_pos[:2]))
            if distance_goal <= 0.5:
                break

            if laser_range_data is not None and laser_angle_data is not None:
                raw_range_data, raw_angle_data = readSensorData() 
                
            laser_data = np.array([raw_angle_data, raw_range_data]).T
            laser_transform = Rz(np.pi / 2)
            noisy_data = get_all_laser_data_ruido(laser_data, laser_transform)

            robot_cell = world_to_grid(np.array(robot_pos), map_size, cell_size)
            Hlw = transformacao(robot_pos, robot_ori)

            obs = []
            obs_pts = []
            obstacle_detected = False

            for i in range(len(laser_data)):
                ang, dist = laser_data[i]

                if (5 - dist) > 0.1:
                    x = dist * np.cos(ang)
                    y = dist * np.sin(ang)
                    point = np.array([x, y, 0, 1])

                    if len(Hlw) != 0:
                        point = Hlw @ point
                        obs.append(point[0:2]) 
                        obs_pts.append(point)
                        
                        grid_point = world_to_grid(point, map_size, cell_size)
                        if 0 <= grid_point[0] < rows and 0 <= grid_point[1] < cols:
                            occupancy_grid = occupancy_grid_mapping(robot_cell, grid_point, occupancy_grid)
                            obstacle_detected = True
                            
                        laser_points.append(point[:2])

            if not obstacle_detected:
                empty_cell = robot_cell + np.array([5 / cell_size, 0])
                occupancy_grid = occupancy_grid_mapping(robot_cell, empty_cell, occupancy_grid)

            Ft, Fatt, Frep = tt_force(np.array(robot_pos[:2]), np.array(goal_position[:2]), laser_data, obs, obs_pts, Hlw)

            dx, dy = Ft
            kr = 0.05
            kt = 0.1

            v = kr * (dx * np.cos(robot_ori[2]) + dy * np.sin(robot_ori[2]))
            w = kt * (np.arctan2(dy, dx) - robot_ori[2])

            v = max(min(v, 0.2), -0.2)
            w = max(min(w, np.deg2rad(45)), -np.deg2rad(45))

            vr = (2 * v + 0.23 * w) / (2 * 0.035)
            vl = (2 * v - 0.23 * w) / (2 * 0.035)

            robot_controller.set_left_wheel_velocity(vl)
            robot_controller.set_right_wheel_velocity(vr)
        
        cont += 1

    robot_controller.set_left_wheel_velocity(0)
    robot_controller.set_right_wheel_velocity(0)
    robot_controller.stop_simulation()
    print('Program ended')

    prob_grid = inv_log_odds(occupancy_grid)
    #print("Valores do occupancy grid:")
    #print(prob_grid)
    
else:
    print("Failed to connect to remote API server")


Program Started
Goal position: [0.4, -4.45, 1] Contador:  0


C:\Users\felip\AppData\Local\Temp\ipykernel_7516\769279799.py:3: RuntimeWarning: divide by zero encountered in log
  return np.log(prob / (1 - prob))
C:\Users\felip\AppData\Local\Temp\ipykernel_7516\769279799.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  return np.log(prob / (1 - prob))


Goal position: [0.8, -1.8, 1] Contador:  1
Goal position: [1.7, 0, 1] Contador:  2
Goal position: [0.7, 3, 1] Contador:  3
Goal position: [2.5, 4.3, 1] Contador:  4
Program ended


### Navegação com ruído

In [7]:
def add_noise(data, mean=0, std_dev=0.01):
    """
    Adiciona ruído gaussiano aos dados.

    Parâmetros:
    - data: Dados originais (numpy array)
    - mean: Média do ruído (padrão é 0)
    - std_dev: Desvio padrão do ruído (padrão é 0.01)

    Retorna:
    - Dados com ruído adicionado
    """
    noise = np.random.normal(mean, std_dev, data.shape)
    return data + noise

# Criando o grid
map_size = np.array([10, 10])
cell_size = 0.1
rows, cols = (map_size / cell_size).astype(int)

# Inicializando o grid
occupancy_grid = np.full((rows, cols), 0.5)

# Conectando-se ao CoppeliaSim
clientID = sim.startSimulation()

if clientID != -1:
    print('Program Started')
    
    robot_controller = RobotController('kobuki')
    robot_controller.start_simulation()
    
    goal_positions = [[0.4, -4.45, 1], [0.8, -1.8, 1], [1.7, 0, 1], [0.7, 3, 1], [2.5, 4.3, 1]] 
    laser_range_data = "hokuyo_range_data"
    laser_angle_data = "hokuyo_angle_data"
    
    laser_points = []
    cont = 0

    while cont < len(goal_positions):
        print('Goal position:', goal_positions[cont], 'Contador: ', cont)
        goal_position = goal_positions[cont]
        robot_controller.set_goal_position(goal_position) 

        while True:
            robot_pos = robot_controller.get_robot_position()
            robot_ori = robot_controller.get_robot_orientation()

            distance_goal = np.linalg.norm(np.array(goal_position[:2]) - np.array(robot_pos[:2]))
            if distance_goal <= 0.5:
                break

            if laser_range_data is not None and laser_angle_data is not None:
                raw_range_data, raw_angle_data = readSensorData() 
                
            laser_data = np.array([raw_angle_data, raw_range_data]).T
            laser_transform = Rz(np.pi / 2)
            noisy_data = add_noise(laser_data)  # Adicionando ruído aos dados do laser

            robot_cell = world_to_grid(np.array(robot_pos), map_size, cell_size)
            Hlw = transformacao(robot_pos, robot_ori)

            obs = []
            obs_pts = []
            obstacle_detected = False

            for i in range(len(noisy_data)):
                ang, dist = noisy_data[i]

                if (5 - dist) > 0.1:
                    x = dist * np.cos(ang)
                    y = dist * np.sin(ang)
                    point = np.array([x, y, 0, 1])

                    if len(Hlw) != 0:
                        point = Hlw @ point
                        obs.append(point[0:2]) 
                        obs_pts.append(point)
                        
                        grid_point = world_to_grid(point, map_size, cell_size)
                        if 0 <= grid_point[0] < rows and 0 <= grid_point[1] < cols:
                            occupancy_grid = occupancy_grid_mapping(robot_cell, grid_point, occupancy_grid)
                            obstacle_detected = True
                            
                        laser_points.append(point[:2])

            if not obstacle_detected:
                empty_cell = robot_cell + np.array([5 / cell_size, 0])
                occupancy_grid = occupancy_grid_mapping(robot_cell, empty_cell, occupancy_grid)

            Ft, Fatt, Frep = tt_force(np.array(robot_pos[:2]), np.array(goal_position[:2]), noisy_data, obs, obs_pts, Hlw)

            dx, dy = Ft
            kr = 0.05
            kt = 0.1

            v = kr * (dx * np.cos(robot_ori[2]) + dy * np.sin(robot_ori[2]))
            w = kt * (np.arctan2(dy, dx) - robot_ori[2])

            v = max(min(v, 0.2), -0.2)
            w = max(min(w, np.deg2rad(45)), -np.deg2rad(45))

            vr = (2 * v + 0.23 * w) / (2 * 0.035)
            vl = (2 * v - 0.23 * w) / (2 * 0.035)

            robot_controller.set_left_wheel_velocity(vl)
            robot_controller.set_right_wheel_velocity(vr)
        
        cont += 1

    robot_controller.set_left_wheel_velocity(0)
    robot_controller.set_right_wheel_velocity(0)
    robot_controller.stop_simulation()
    print('Program ended')

    prob_grid = inv_log_odds(occupancy_grid)
    #print("Valores do occupancy grid:")
    #print(prob_grid)
    
else:
    print("Failed to connect to remote API server")

Program Started
Goal position: [0.4, -4.45, 1] Contador:  0


C:\Users\felip\AppData\Local\Temp\ipykernel_7516\769279799.py:3: RuntimeWarning: divide by zero encountered in log
  return np.log(prob / (1 - prob))
C:\Users\felip\AppData\Local\Temp\ipykernel_7516\769279799.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  return np.log(prob / (1 - prob))


Goal position: [0.8, -1.8, 1] Contador:  1
Goal position: [1.7, 0, 1] Contador:  2
Goal position: [0.7, 3, 1] Contador:  3
Goal position: [2.5, 4.3, 1] Contador:  4
Program ended


### Salvar imagens

In [8]:
# Funções para salvar as figuras
def save_occupancy_grid(occupancy_grid, filename):
    """
    Salva o occupancy grid como uma imagem.

    Parâmetros:
    - occupancy_grid: Grid de ocupação (numpy array)
    - filename: Nome do arquivo para salvar a imagem
    """
    plt.imshow(np.rot90(occupancy_grid), cmap='gray_r')
    plt.title("Occupancy Grid Map")
    plt.savefig(filename)
    plt.close()
    print(f"Occupancy grid saved as {filename}")

def save_laser_points(points, filename):
    """
    Salva os pontos do laser como uma imagem.

    Parâmetros:
    - points: Lista de pontos do laser (lista de numpy arrays)
    - filename: Nome do arquivo para salvar a imagem
    """
    points = np.array(points)
    plt.scatter(points[:, 0], points[:, 1], c='#2176b5', s=1)
    plt.grid(False)
    plt.axis('equal')
    plt.title("Laser Points")
    plt.savefig(filename)
    plt.close()
    print(f"Laser points saved as {filename}")
    
# Salvar o occupancy grid
save_occupancy_grid(occupancy_grid, "occupancy_grid_ruido.png")

# Salvar os pontos do laser
save_laser_points(laser_points, "laser_points_ruido.png")

Occupancy grid saved as occupancy_grid_ruido.png
Laser points saved as laser_points_ruido.png
